In [2]:

import torch
import torch.multiprocessing

from helpers.tools import *
from helpers.tools import lagged_batch_corr
from model.model_wrapper import Architecture_PL

torch.multiprocessing.set_sharing_strategy('file_system')

# Loads all checkpoints in a given folder structure and calculates all necessary predictions. Also saves the according labels to be save.

# For zero shot we test three things: 

- River Benchmark
- Aerosol Benchmark

Selected this here by hand because I am very lazy

In [33]:
auroc = torchmetrics.classification.BinaryAUROC()

In [34]:
# Load the best models: 
best = pd.read_csv("multirun/experiment_2_std_big/summary.csv", index_col=0).T

stack = []
for x in best["model.model_type"].unique():
    stack.append(best.loc[best["model.model_type"] == x].sort_values("Test1_Auroc", ascending=False)[:1])

pd.concat(stack)

HP,seed,hparams_search,model.optimizer_lr,data.batch_size,model.weight_decay,model.model_type,model.corr_regularization,corr_input,regression_head,data.ds_name,max_lags,n_vars,model,Test1_Auroc,Test1_MSE,Test2_AUROC,Test2_MSE
6,43,std_runs_slurm.yaml,0.001,16,0.01,convM,True,True,False,joint,3,5,big.yaml,0.871109,0.03637622,0.89227587,0.03218321
9.1,50,std_runs_slurm.yaml,0.001,64,0,transformer,False,True,False,joint,3,5,big.yaml,0.971022,0.012118887,0.97848606,0.009607594
7.2,49,std_runs_slurm.yaml,0.001,16,0.01,bidirectional,False,True,False,joint,3,5,big.yaml,0.96785104,0.013725153,0.97649235,0.009861699
4.3,42,std_runs_slurm.yaml,0.0001,64,0,mlp,True,True,False,joint,3,5,big.yaml,0.791368,0.046149682,0.7982332,0.043174013
8.4,50,std_runs_slurm.yaml,0.001,64,0.01,unidirectional,False,True,False,joint,3,5,big.yaml,0.96973234,0.013177577,0.9781359,0.009636639


In [35]:
mlp = "multirun/experiment_2_std_big/13-47-18/4/run/epoch=13-step=6902.ckpt"
uni = "multirun/experiment_2_std_big/13-43-18/8/run/epoch=57-step=57130.ckpt"
bi= "multirun/experiment_2_std_big/13-45-14/7/run/epoch=35-step=141768.ckpt"
conv = "multirun/experiment_2_std_big/13-46-17/6/run/epoch=7-step=31504.ckpt"
trf = "multirun/experiment_2_std_big/13-44-13/9/run/epoch=65-step=32538.ckpt"
best = [mlp, uni, bi, conv, trf]

# River

In [36]:
res = []
data = load_river_data()
# create proper padding
a= torch.concat([data[0][0,:,:], torch.normal(0, 0.1, (len(data[0][0]),2))], axis=1)
a = a.unsqueeze(0)
corr = lagged_batch_corr(a,3)


for x in best:
    model = Architecture_PL.load_from_checkpoint(x)
    M = model.model
    M = M.to("cpu")
    M = M.eval()
    pred = torch.sigmoid(M((a[:,:600,:], corr)))[0,:3,:3, -1]
    res.append(auroc(pred, data[1]))

In [38]:
pd.DataFrame(res, index = ["MLP", "uGRU", "bGRU", "CM", "Trf"])

,0
MLP,tensor(0.4250)
uGRU,tensor(1.0000)
bGRU,tensor(0.5000)
CM,tensor(0.6000)
Trf,tensor(1.0000)


In [37]:
pd.DataFrame(res, index = ["MLP", "uGRU", "bGRU", "CM", "Trf"]).to_csv("scores/zero_shot_rivers.csv")

# Kuramoto

In [39]:
data = torch.load("data/deterministic_ds/kuramoto/simple_test.p")

In [10]:
X = torch.stack([x[0] for x in data])[:250]
Y = torch.stack([x[1] for x in data])[:250]
X = X - X.min() / (X.max() - X.min())

In [11]:
corr = lagged_batch_corr(X,3)

In [12]:
res = []
cleaned_res = []
for x in best:
    model = Architecture_PL.load_from_checkpoint(x)
    M = model.model
    M = M.to("cpu")
    M = M.eval()
    #pred = torch.sigmoid(M((X, corr)))[:,:,:,-1]
    pred = torch.sigmoid(M((X, corr))).max(axis=3)[0]

    res.append(auroc(pred, Y))
    stack_pred_cleaned = []
    stack_lab_cleaned = []
    for x in pred:
        stack_pred_cleaned.append(x.flatten()[~ torch.eye(5,5).flatten().bool()])
    for x in Y:
        stack_lab_cleaned.append(x.flatten()[~ torch.eye(5,5).flatten().bool()])
    cleaned_res.append(auroc(torch.concat(stack_pred_cleaned), torch.concat(stack_lab_cleaned)))

In [13]:
res,cleaned_res

([tensor(0.4665),
  tensor(0.4403),
  tensor(0.4183),
  tensor(0.4230),
  tensor(0.4227)],
 [tensor(0.5018),
  tensor(0.5466),
  tensor(0.4962),
  tensor(0.5086),
  tensor(0.5939)])

In [14]:
auroc(torch.concat(stack_pred_cleaned), torch.concat(stack_lab_cleaned))

tensor(0.5939)

# Aerosol

In [3]:
data = pickle.load(open("data/deterministic_ds/aerosol/simple_test.p", "rb"))

In [4]:
X =data[0][:300]
Y = data[1][:300]
X = (X - X.min()) / (X.max() - X.min())
corr = lagged_batch_corr(X,3)

In [5]:
res = []
for x in best:
    model = Architecture_PL.load_from_checkpoint(x)
    M = model.model
    M = M.to("cpu")
    M = M.eval()
    pred = torch.sigmoid(M((X, corr)))[:,:,:,-1]
    #pred = torch.sigmoid(M((X, corr))).max(axis=3)[0]
    res.append(pred[:,0,1] > pred[:,1,0])
pd.DataFrame([(x.sum() / len(x)).numpy() for x in res], index = ["MLP", "uGRU", "bGRU", "CM", "Trf"]).T

NameError: name 'best' is not defined